Code to select a median traveltime for each quarry-station pair

In [40]:
import pandas as pd
import SeisBlast as sb
import numpy as np
#import matplotlib.pyplot as plt
seisblast_files = '/home/bmelo/bin/SeisBlast/files'

##Import necessary files: Catalogue, Relocated Catalogue, Stations locations##


In [41]:
james_catalogue = pd.read_csv('/mnt/REPO/QUARRY-BLASTS/CATALOGS/quarry-blast-catalog-2013-2014.csv', sep = r'\s+')
james_bulletins_dir = "/mnt/REPO/QUARRY-BLASTS/BULLETINS/"
stations = pd.read_csv(
    "/home/bmelo/bin/Quarry/James_files/Total_Stations_List.txt", 
    delimiter=' ',
    header = 0,
    names=['sta', 'name','lat', 'lon', 'elevation', 'net', 'end'])
#to do in the future: make one catalogue with both locations, before and after relocation
relocated_blasts = pd.read_csv('/home/bmelo/bin/SeisBlast/files/Eire_blasts_relocated.csv')
#merge with the relocated quarry blasts lists to get the relocated events
relocated_catalogue = pd.merge(james_catalogue, relocated_blasts, left_on='EVENT-ID', right_on='id-ev')
relocated_catalogue = relocated_catalogue.sort_values(by=['lon-quarry', 'lat-quarry']).reset_index(drop=True)
relocated_catalogue.head(5)

,EVENT-ID,DATE,TIME,LAT(deg),LAT-ERR(km),LON(deg),LON-ERR(km),DEPTH(km),RMS(s),AZ-GAP(deg),N_PHASES,MAG(ML),lon_ev,lat_ev,id-ev,lon-quarry,lat-quarry
0,dias2013ufgm,2013-10-15,11:40:26.676,51.98154,1.445,-10.21300,2.064,0.0,0.713,163.6,24,1.30,-10.21300,51.98154,dias2013ufgm,-10.177543,51.952300
1,dias2014gote,2014-04-04,10:48:24.192,51.97051,1.711,-10.19241,1.919,0.0,0.761,141.8,22,1.25,-10.19241,51.97051,dias2014gote,-10.177543,51.952300
2,dias2014sjxg,2014-09-19,13:37:28.010,51.97509,1.767,-10.18196,1.883,0.0,0.552,151.3,21,1.32,-10.18196,51.97509,dias2014sjxg,-10.177543,51.952300
3,dias2014gnas,2014-04-03,12:20:07.582,51.90990,2.610,-10.07899,1.920,0.0,0.443,191.3,17,1.11,-10.07899,51.90990,dias2014gnas,-10.069618,51.892000
4,dias2014iyzq,2014-05-08,12:30:09.045,53.51024,1.814,-10.05044,3.756,0.0,0.732,249.4,18,1.38,-10.05044,53.51024,dias2014iyzq,-10.042922,53.501868


##Organize data so we can have a list of travel time at each station-quarry pair##

In [ ]:
quarry_data = pd.DataFrame()

#first loop over the values of the relocated catalogue
for index, row in relocated_catalogue.iterrows():
    #qry_id = row['quarry-ID']
    ev_id = row['EVENT-ID']
    #get the picks for the event at the index position
    df, info = sb.list_picks(james_catalogue, james_bulletins_dir, ev_id)
    #calculate the travel time in seconds from the origin time
    origin_time = pd.to_datetime((relocated_catalogue.loc[index, 'DATE'] + ' ' + relocated_catalogue.loc[index, 'TIME']), format='%Y-%m-%d %H:%M:%S.%f')
    df['Ptime(s)'] = (df['time']['P'] - origin_time).dt.total_seconds()
    #new dataframe to store the quarry data for the event
    temp_df = pd.DataFrame({
    'quarry-lat': row['lat-quarry'],
    'quarry-lon': row['lon-quarry'],
    'sta': df['sta'], 
    'Ptime(s)': df['Ptime(s)'],
    'ev_id': ev_id,
    'dist_def': df['dist'],
    })
    #populate quarry_data_group
    quarry_data = pd.concat([quarry_data, temp_df], axis=0, ignore_index=True)

quarry_data.head(5)

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id
0,51.9523,-10.177543,IA003,31.984,dias2013ufgm
1,51.9523,-10.177543,IA004,26.468,dias2013ufgm
2,51.9523,-10.177543,IA007,30.338,dias2013ufgm
3,51.9523,-10.177543,IA009,36.326,dias2013ufgm
4,51.9523,-10.177543,IAD23,28.709,dias2013ufgm


In [43]:
#group by quarry ID and station and calculate the median, std and count of the picks
quarry_data_group = quarry_data.dropna().groupby(['quarry-lat','quarry-lon','sta']).agg({'Ptime(s)': list, 'ev_id': list}).reset_index()
#quarry_data_group['Ptime(s)'] = quarry_data_group['Ptime(s)'].apply(lambda x: x if isinstance(x, list) else [x])
quarry_data_group


,quarry-lat,quarry-lon,sta,Ptime(s),ev_id
0,51.8119,-8.654800,IA003,"[24.938, 25.08]","[dias2014igwd, dias2014kwjz]"
1,51.8119,-8.654800,IA005,"[8.251, 7.974]","[dias2014igwd, dias2014kwjz]"
2,51.8119,-8.654800,IA006,[13.44],[dias2014kwjz]
3,51.8119,-8.654800,IA007,[15.282],[dias2014igwd]
4,51.8119,-8.654800,IA012,"[28.109, 28.117]","[dias2014igwd, dias2014kwjz]"
...,...,...,...,...,...
5723,55.2586,-7.205738,DL31,[6.315],[dias2014bqzf]
5724,55.2586,-7.205738,IAD33,[20.536],[dias2014gwcc]
5725,55.2586,-7.205738,IDGL,"[4.959, 4.905, 4.495]","[dias2014bqzf, dias2014gwcc, dias2014thky]"
5726,55.2586,-7.205738,ILTH,[24.762],[dias2014thky]


In [44]:
#calculate the median, std and count of the picks
quarry_data_group['median'] = quarry_data_group['Ptime(s)'].apply(lambda x: np.median(x))
quarry_data_group['std'] = quarry_data_group['Ptime(s)'].apply(lambda x: sum((i - sum(x)/len(x))**2 for i in x)/len(x))
quarry_data_group['count'] = quarry_data_group['Ptime(s)'].apply(lambda x: len(x))
quarry_data_group.head(10)

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id,median,std,count
0,51.8119,-8.6548,IA003,"[24.938, 25.08]","[dias2014igwd, dias2014kwjz]",25.0090,0.005041,2
1,51.8119,-8.6548,IA005,"[8.251, 7.974]","[dias2014igwd, dias2014kwjz]",8.1125,0.019182,2
2,51.8119,-8.6548,IA006,[13.44],[dias2014kwjz],13.4400,0.000000,1
3,51.8119,-8.6548,IA007,[15.282],[dias2014igwd],15.2820,0.000000,1
4,51.8119,-8.6548,IA012,"[28.109, 28.117]","[dias2014igwd, dias2014kwjz]",28.1130,0.000016,2
5,51.8119,-8.6548,IAD01,"[20.388, 20.424]","[dias2014igwd, dias2014kwjz]",20.4060,0.000324,2
6,51.8119,-8.6548,IAD23,[19.928],[dias2014igwd],19.9280,0.000000,1
7,51.8119,-8.6548,IAD38,"[4.587, 4.482, 4.533]","[dias2014igwd, dias2014kwjz, dias2014ushl]",4.5330,0.001838,3
8,51.8119,-8.6548,IGLA,"[30.445, 30.617]","[dias2014kwjz, dias2014ushl]",30.5310,0.007396,2
9,51.8119,-8.6548,IWEX,[23.831],[dias2014igwd],23.8310,0.000000,1


##select event with travel time closest to the mean##

In [ ]:
quarry_data_group['closest-time'] = quarry_data_group.apply(lambda row: min(row['Ptime(s)'], key=lambda x: abs(x-row['median'])), axis=1)
quarry_data_group['closest-ev_id'] = quarry_data_group.apply(lambda row: row['ev_id'][row['Ptime(s)'].index(row['closest-time'])], axis=1)
# #now i need to add information about the origin time and magnitude of the event
quarry_data_group['origin_time'] = quarry_data_group['closest-ev_id'].apply(lambda x: relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'DATE'].values[0] + ' ' + relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'TIME'].values[0])
quarry_data_group['mag'] = quarry_data_group['closest-ev_id'].apply(lambda x: relocated_catalogue.loc[relocated_catalogue['EVENT-ID'] == x, 'MAG(ML)'].values[0])
quarry_data_group

,quarry-lat,quarry-lon,sta,Ptime(s),ev_id,median,std,count,closest-time,closest-ev_id,origin_time,mag
0,51.8119,-8.654800,IA003,"[24.938, 25.08]","[dias2014igwd, dias2014kwjz]",25.0090,0.005041,2,25.080,dias2014kwjz,2014-06-04 13:23:17.921,1.39
1,51.8119,-8.654800,IA005,"[8.251, 7.974]","[dias2014igwd, dias2014kwjz]",8.1125,0.019182,2,8.251,dias2014igwd,2014-04-28 14:17:33.523,1.26
2,51.8119,-8.654800,IA006,[13.44],[dias2014kwjz],13.4400,0.000000,1,13.440,dias2014kwjz,2014-06-04 13:23:17.921,1.39
3,51.8119,-8.654800,IA007,[15.282],[dias2014igwd],15.2820,0.000000,1,15.282,dias2014igwd,2014-04-28 14:17:33.523,1.26
4,51.8119,-8.654800,IA012,"[28.109, 28.117]","[dias2014igwd, dias2014kwjz]",28.1130,0.000016,2,28.109,dias2014igwd,2014-04-28 14:17:33.523,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...
5723,55.2586,-7.205738,DL31,[6.315],[dias2014bqzf],6.3150,0.000000,1,6.315,dias2014bqzf,2014-01-24 12:27:29.676,0.76
5724,55.2586,-7.205738,IAD33,[20.536],[dias2014gwcc],20.5360,0.000000,1,20.536,dias2014gwcc,2014-04-08 11:16:03.644,0.78
5725,55.2586,-7.205738,IDGL,"[4.959, 4.905, 4.495]","[dias2014bqzf, dias2014gwcc, dias2014thky]",4.9050,0.042924,3,4.905,dias2014gwcc,2014-04-08 11:16:03.644,0.78
5726,55.2586,-7.205738,ILTH,[24.762],[dias2014thky],24.7620,0.000000,1,24.762,dias2014thky,2014-10-02 10:39:38.805,0.58


In [62]:
#group events to have a station list for each quarry
quarry_data_ev = quarry_data_group.groupby(['closest-ev_id','quarry-lat','quarry-lon','origin_time','mag']).agg({'sta': list, 'closest-time': list}).reset_index()
quarry_data_ev['count'] = quarry_data_group['sta'].apply(lambda x: len(x))
quarry_data_ev.to_csv(f'{seisblast_files}/uniq_quarry-station_list.csv', index=False)
quarry_data_ev['sta'].iloc[3][1]

'UHELL'